テンソル：Tensors 
==========================

Pytorchではテンソル型の変数を利用して入力や出力，モデルのパラメータを表現する．

テンソルはNumPyのndarraysに似ているが、違いとしてGPUや他のハードウェアアクセラレータ上で動作させることが可能．

また，テンソルは自動微分に最適化されている（後述する）

NumPyのndarraysに触れたことのある人は飛ばしてこの章はok．

触れたことのない，または自信がない人はおすすめ.

また，他の章をこなした後にわからないところを見にくるのもいい．(自分も何度も見直してる)

In [1]:
import torch
import numpy as np

テンソルは様々な方法で作成可能．

1.データから直接テンソルに変換

In [2]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
print(x_data)

tensor([[1, 2],
        [3, 4]])


2.Numpy arraysからテンソルに変換

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])


3.他のテンソルの形を真似て作成．

In [4]:
x_ones = torch.ones_like(x_data)
print(x_ones)

tensor([[1, 1],
        [1, 1]])


In [5]:
x_random = torch.rand_like(x_data,dtype=torch.float)
print(x_random)

tensor([[0.5430, 0.6831],
        [0.6221, 0.8972]])


4.shapeを指定してテンソル作成

In [6]:
shape = (2,3)
rand = torch.rand(shape)
ones = torch.ones(shape)
zeros = torch.zeros(shape)
print(rand,"\n",ones,"\n",zeros)

tensor([[0.0290, 0.0914, 0.3842],
        [0.3972, 0.7503, 0.4090]]) 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


テンソルは属性変数として、その形状、データの型、保存されているデバイスを保持している．

サイズを確認したいときに，.shapeを使うのはよく使う


In [7]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


GPUの操作
==================

PyTorchでは、算術、線形代数、行列操作（転置、インデックス、スライス）など、100種類以上のテンソル演算が可能．

これらの各操作をGPU上で行うことが可能なのが深層学習ライブラリの特徴の一つ．

デフォルト(上記のテンソルたち)ではテンソルはCPU上に作られる．

主に以下の二つのメソッドでテンソルをGPU上に移動させる．


 Google ColabでGPUを使うためにはランタイムの接続方法を変更する必要がある．

 やり方は簡単で「ランタイム」→「ランタイムのタイプを変更」→「None」→「GPU」とすれば良い．

1..toメソッド

In [8]:
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
a = torch.ones(shape).to(device)
print(a)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


2.cudaメソッド

In [9]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.empty_cache()
    torch.cuda.set_device(0)
b = torch.ones(shape).cuda()
print(b)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


どちらのメソッドでも構わないが過去に自分は1を使っていて何らかの不都合が生じて(詳細は覚えてない)以来2を使っているのでこだわりがなければ2をおすすめする．

テンソルの算術計算
==================
Tensor APIの算術計算の記法はNumPy APIを踏襲している．


In [10]:
data = [1.,2,3,4]

In [11]:
tensor = torch.tensor(data)
print(tensor)

tensor([1., 2., 3., 4.])


1.要素ごとの積を求めるには次の三つのどれかを使う

  おそらく各手法で微妙に計算時間が変わると思うが微小なので気にしなくて良い(変わったらごめん)

  おすすめはtensor*tensor

In [12]:
a1 = tensor * tensor
a2 = tensor.mul(tensor)

_ = torch.rand_like(tensor)
a3 = torch.mul(tensor, tensor, out=_)

print(a1 - a2)

print(a1 - a3)

tensor([0., 0., 0., 0.])
tensor([0., 0., 0., 0.])


2.行列の掛け算を求める場合は次の三つを使う．

上に同じ

おすすめはtensor @ tensor.T

In [13]:
b1 = tensor @ tensor.T
b2 = tensor.matmul(tensor.T)

__ = torch.rand_like(tensor)
b3 = torch.matmul(tensor, tensor.T, out=__)

print(b1 - b2)
print(b1 - b3)

tensor(0.)
tensor(0.)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: An output with one or more elements was resized since it had shape [4], which does not match the required output shape []. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  """


**1要素のテンソル** 

1要素のテンソル（テンソルの全要素を足し算する等をした結果生まれる）を扱う場合には、``.item()``を使用する.

テンソル型ではなくpythonの数値型として取り出せる

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg)
print(agg_item, type(agg_item))